__применение парсера объявлений циан.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
import sys
import re
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from random import random

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [3]:
raw_data_path = 'data/raw/flat/cian' # каталог с файлами объявлений

In [4]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'
# base_url, req_param = 'https://sevastopol.cian.ru/kupit-kvartiru/', ''
base_url, req_param ='https://www.cian.ru/cat.php', 'deal_type=sale&offer_type=flat&region=184723'

In [5]:
%%time

# from lib.downloader import DownloaderSimple
# from lib.parser import AdsListParser
from lib.downloader import DownloaderSeleniumFirefox
from lib.cian import CianParser


df,html = CianParser(
        base_url = base_url,
        driver=DownloaderSeleniumFirefox(profile_path),
        # driver=DownloaderSimple(),
    ).load(req_param=req_param, keep_html=True) # , npage_start=53) # , page_limit=3 ) #

print(len(df))

[INFO    ] 2022-09-12 13:49:36 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-12 13:49:36 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-12 13:49:38 | AdsListParser: downloader init
[INFO    ] 2022-09-12 13:49:38 | CianParser: downloader init
[INFO    ] 2022-09-12 13:49:38 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-12 13:49:54 | AdsListParser: read page 1
[INFO    ] 2022-09-12 13:50:08 | AdsListParser: read page 2
[INFO    ] 2022-09-12 13:50:22 | AdsListParser: read page 3
[INFO    ] 2022-09-12 13:50:27 | AdsListParser: read page 4
[INFO    ] 2022-09-12 13:50:44 | AdsListParser: read page 5
[INFO    ] 2022-09-12 13:50:50 | AdsListParser: read page 6
[INFO    ] 2022-09-12 13:51:06 | AdsListParser: read page 7
[INFO    ] 2022-09-12 13:51:17 | AdsListParser: read page 8
[INFO    ] 2022-09-12 13:51:22 | AdsListParser: read page 9
[INFO    ] 2022-09-12 13:51:33 | AdsListParser: read page 10
[INFO    ] 2022-09-12 13:51:44 | AdsLis

In [6]:
df.sample(3)

,OfferTitle,OfferSubtitle,Deadline,MainPrice,PriceInfo,GeoLabel,TimeLabel,LinkArea,Description,page,ts
1271,"1-комн. апарт., 28 м², 3/3 этаж",,,3 700 000 ₽,132 143 ₽/м²,"[Севастополь, р-н Гагаринский, мкр. Омега, улица Челнокова, 49]","[четыре дня назад, 8 сен, 09:06]",https://sevastopol.cian.ru/sale/flat/273875003/,"Сохрани сбережения-купи недвижимость!\nЛиквидные АПАРТАМЕНТЫ в самом привлекательном месте\n г. Севастополя!\nИНВЕСТИЦИИ В НЕДВИЖИМОСТЬ С ГАРАНТИЕЙ!\n\nБУХТА ОМЕГА ПАРК ПОБЕДЫ,МОРЕ, СПОРТИВНАЯ ЗОНА, АКВАПАРК, ДЕТСКАЯ ЗОНА РАЗВЛЕЧЕНИЙ КАФЕ РЕСТОРАНЫ, ДЕЛЬФИНАРИЙ\n\nАпартаменты 28,5 м2.\nВторой этаж Балкон. \n\nГородские коммуникации.\nОбъект сдан!\n\nПолная консультация и сопровождение сделки.\nСтоимость: 3 600 000",46,2022-09-12 13:59:10.149602
125,"3-комн. кв., 134,6 м², 11/12 этаж",,,19 000 000 ₽,141 159 ₽/м²,"[Севастополь, р-н Гагаринский, мкр. Летчики, улица Маячная, 33]","[5 часов назад, сегодня, 08:38]",https://sevastopol.cian.ru/sale/flat/268756772/,"Трехкомнатная двухуровневая квартира 134,6 м2 с площадью кухни 36,3 м2, на одиннадцатом и двенадцатом этаже двенадцатиэтажного дома. Расположена в г. Севастополь, Гагаринский район, улица Маячная. \nПрекрасное предложение для дружной семьи или в качестве инвестиции. Двухуровневая квартира с прекрасным видом на море и город. Если вы цените безопасность, комфорт и хотите жить в лучшем районе города Севастополя - это предложение именно для Вас. \n \nПозвоните нам, чтобы узнать все детали! \n \nТехнические характеристики: \n \n-Адрес: г. Севастополь, Гагаринский район, улица Маячная\n-Год постройки дома: 2016\n-Общая площадь: 134,6 м2\n-Жилая площадь: 75 м2\n-Площадь кухни: 36,3 м2\n-Высота потолка: 3 м\n-Водоснабжение: централизованное\n-Отопление, горячее водоснабжение: АГВ\n-Газоснабжение: есть\n \nДополнительная информация: \nДом 2016 года, благоустроенная придомовая территория. Светлая квартира свободной планировки расположена в тихом районе города с развитой инфраструктурой, рядом пляж и парк.\n \n-На первом уровне квартиры размещены большая кухня-гостиная, санузел, балкон.\n-На втором уровне - холл (возможно разместить рабочее место), три спальни, санузел, гардеробная, два балкона.\nКвартира светлая и теплая, со своим автономным отоплением (АГВ).\nСостояние: черновая отделка, есть разводка электричества, есть дизайн-проект.\n \nПозвоните нам, чтобы назначить просмотр! \n \nИнфраструктура: \nМикрорайон с развитой инфраструктурой: рядом остановка, рынок, аптеки, супермаркет, медицинский центр. В самом доме расположилась стоматологическая клиника, продуктовые магазины.\nB пешей доступности крупнейшего ""Парка Победы"" и несколько скверов, песочный пляж Омега и галечный пляж парка. На автомобиле вы сможете добраться за 17 минут до исторического центра г. Севастополь.\n \nПодходит под ипотеку, материнский капитал.\n \nДокументы проверены юристами и готовы к сделке.\n \nДобавьте предложение в закладки, чтобы не потерять!",5,2022-09-12 13:59:10.149602
428,"1-комн. кв., 49,62 м², 2/10 этаж",Сдача корпуса 4 кв. 2023,сдача ГК: 4 кв. 2023 года,4 217 700 ₽,85 000 ₽/м²,"[Севастополь, р-н Гагаринский, Казачья Бухта мкр, Скифия ЖК]","[2 часа назад, сегодня, 11:11]",https://sevastopol.cian.ru/sale/flat/273181103/,"Прекрасная квартира с городской пропиской в новом доме у моря. \n5 минут до пляжа! Остановка в шаговой доступности , 15 минут до центра города!\n\nДвухконтурный газовый котел. Индивидуальное отопление, которое позволит Вам не только сократить коммунальные платежи, но и создать комфортную температуру в квартире. \nУстановлены приборы учета (газ, вода, электричество)\nВысокие потолки (2.8 м) и увеличенные стеклопакеты визуально делают квартиру еще более светлой и просторной.\n\nТерритория жилого комплекса полностью огорожена. \nУютный зеленый двор с удобными скамейками для отдыха и детская площадка с каруселями и горками, которая обязательно понравится Вашим детям!\n\nМоре и пляж, магазины и социальные объекты (новая школа и детский сад), исторические и культурные о

In [7]:
print(len(df))
df = pd.DataFrame(df).dropna().drop_duplicates(['LinkArea','Description',]).reset_index(drop=True)
print(len(df))

1509
1506


In [8]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
# df.to_excel(f'{raw_data_path}/cian_{ts}_raw.xlsx',index=False)
df.to_pickle(f'{raw_data_path}/cian_{ts}_raw.pkl')

----

In [9]:
# df = pd.read_excel('data/raw/cian/cian_2022-09-08_13-49_raw.xlsx')

In [10]:
# df

In [11]:
# df['GeoLabel'].apply( lambda s : len(s) ) #', '.join(s) )

# s = ['Севастополь', 'р-н Гагаринский', 'Казачья Бухта мкр', 'Скифия ЖК']
# ', '.join(s)

In [12]:
# from lib.cian import CianDataCleaner

# CianDataCleaner().transform(df)

----

In [13]:
# with open('tmp/cian.html','wt') as f: f.write(html[0])